### API Keys

In [ ]:
api_key = "" # vanna api key
GEMINI_API_KEY='' # LLM api key

### Import & installations

In [ ]:
import vanna
from vanna.chromadb import ChromaDB_VectorStore
from vanna.google import GoogleGeminiChat
import google.generativeai as genai
import pyodbc
import pandas as pd
import platform

### Innitiate vanna class with specific vector DB Vector store & LLM

In [ ]:
class MyVanna(ChromaDB_VectorStore, GoogleGeminiChat):
    def __init__(self, config=None):
        ChromaDB_VectorStore.__init__(self, config=config)
        GoogleGeminiChat.__init__(self, config={'api_key': GEMINI_API_KEY, 'model': "gemini-pro"})

vn = MyVanna()

### Connect to DB 

In [ ]:
vn.connect_to_mssql(odbc_conn_str='DRIVER={ODBC Driver 17 for SQL Server};SERVER=---;DATABASE=---;UID=--;PWD=---') 
# You can use the ODBC connection string here


In [ ]:
vn

### Def function to run Query using provided connection

In [ ]:
def run_sql (sql :str) -> pd.DataFrame:
    df = pd.read_sql_query(sql,conn)
    return df

In [ ]:
vn.run_sql = run_sql
vn.run_sql_is_set=True

In [ ]:
conn = pyodbc.connect(driver="{ODBC Driver 17 for SQL Server}",
                      host = platform.node(),
                      database="----", trusted_connection = 'yes'
)

### Capture INFORMATION_SCHEMA to train the vanna model

Part 1 : use DB schema

In [ ]:
# The information schema query may need some tweaking depending on your database. This is a good starting point.
df_information_schema = vn.run_sql("SELECT * FROM INFORMATION_SCHEMA.COLUMNS")



In [ ]:
# This will break up the information schema into bite-sized chunks that can be referenced by the LLM
plan = vn.get_training_plan_generic(df_information_schema)
plan



In [ ]:
# If you like the plan, then run to train
vn.train(plan=plan)

Part 2 : use of DDL statments

In [ ]:
# The following are methods for adding training data. Make sure you modify the examples to match your database.

# DDL statements are powerful because they specify table names, colume names, types, and potentially relationships
vn.train(ddl="""
    CREATE TABLE IF NOT EXISTS my-table (
        id INT PRIMARY KEY,
        name VARCHAR(100),
        age INT
    )
""")



Part 3 : use of Documentation to define the cases which are not covered in above training or any other cases which needs extra attention  

In [ ]:
# Sometimes you may want to add documentation about your business terminology or definitions.
vn.train(documentation="Our business defines OTIF score as the percentage of orders that are delivered on time and in full")



Part 4 : use of existing / important set of SQL queries to train 

In [ ]:
# You can also add SQL queries to your training data. This is useful if you have some queries already laying around. You can just copy and paste those from your editor to begin generating new SQL.
vn.train(sql="SELECT * FROM my-table WHERE name = 'John Doe'")


### Creating training data & method to remove specific training data

In [ ]:
# At any time you can inspect what training data the package is able to reference
training_data = vn.get_training_data()
training_data
# You can remove training data if there's obsolete/incorrect information. 
#vn.remove_training_data(id='1-ddl')


### Ask Questions to the model

In [ ]:
## Asking the AI

vn.ask(question="what is the total count of orders")



### Use below built in Flask app to interact & fine-tune the model

In [ ]:
from vanna.flask import VannaFlaskApp
app = VannaFlaskApp(vn)
app.run()